# Hand motions classification using non-invasive EEG recordings
### by Cedric Simar and Antoine Passemiers
<hr/>

## Table of content

* [0 - Introduction](#introduction)
  * [0.1. Problem description](#problem-description)


* [1 - Preprocessing](#preprocessing)
  * [1.1. Import useful libraries](#import-libraries)
  * [1.2. Load the data](#load-data)
  * [1.3. Low-pass filtering](#low-pass-filtering)
  * [1.4. Downsampling](#downsampling)


* [2 - A first naive approach: learn from raw EEG samples](#learn-from-raw-data)
  * [2.1. Random Forests (first model)](#random-forests)
    * [2.1.1 VC-dimension of random forests](#random-forests-vc)
    * [2.1.2 Random forest pruning](#random-forest-pruning)
  * [2.2  Logistic regression (second model)](#logistic-regression)
    * [2.2.1 VC dimension of logistic regression](#logistic-regression-vc)


* [3. Riemannian-based kernel trick](#kernel-trick)
  * [3.1. Riemannian-based SVM (fourth model)](#ghmm)
  * [3.2. Linear Discriminant Analysis (fifth model)](#lda)


* [4 - Dealing with non-stationary data](#non-stationary-data)
  * [4.1. Gaussian Hidden Markov Models (third model)](#ghmm)


* [5. ANN, LSTM and weird stuff](#CAFE)


* [X. Bibliography](#bibliography)

## Introduction <a class="anchor" id="introduction"></a>
<hr/>

Note: This notebook has been designed in order to serve several purposes:
1. Introduce different models either that have been successfully applied to EEG-based problems in the past or that are considered by us as being relevant given the nature of the task.
2. Constantly refer to generalization theory while selecting our best models using Structural Risk Minimization (SRM)
3. Present code excerpts used to achieve good performance on the [Kaggle competition](#bib-kaggle) about Grasp-and-Lift EEG detection.

### Problem description <a class="anchor" id="problem-description"></a>

Some patients have lost the use of one of their hand, and this may be due either by neurological handicaps or an amputation.
The research goal of the present competition is to give those patients their full autonomy back by developing BCI (Brain-Computer Interface) devices capable of playing the role of a hand. This requires to design algorithms capable of mapping brain waves to the desired movement. In the framework of this competition, there are 6 different movements that one must be able to identify: 'HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', and 'BothReleased'.

In [1]:
from IPython.display import HTML
url = "https://www.youtube.com/embed/y3_Izuop2gY?rel=0&amp;controls=0&amp;showinfo=0"
HTML('<iframe style="display:block" width="560" height="315" src="%s" frameborder="0" allowfullscreen></iframe>' % url)

## Preprocessing <a class="anchor" id="preprocessing"></a>
<hr/>

### Import useful libraries <a class="anchor" id="import-libraries"></a>

In [1]:
%load_ext Cython

import cython
import os
import gc
import time
import scipy.linalg
import scipy.signal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

### Load the data <a class="anchor" id="load-data"></a>

TODO Cedric: descriptions (electrodes, multi-labels, échantillonage, tout ça tout ça)

<figure style="text-align:center;">
  <img src="imgs/EEG_Electrode_Numbering.jpg" style="width:450px;">
  <figcaption> Source: [Kaggle](#bib-kaggle) </figcaption>
</figure>

In [2]:
N_PATIENTS = 12
# Feature names
ELECTRODE_NAMES = [
    'Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
    'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2', 'CP6', 'TP10',
    'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz', 'O2', 'PO10']
# Label names
EVENT_NAMES = ['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase', 'LiftOff', 'Replace', 'BothReleased']


def load_dataset(subject=1, data_dir='', series=range(1,9)):
    data, events = list(), list()
    for series_id, s in enumerate(series):
        print("Load series {0} from patient {1}".format(series_id+1, subject))
        data_filepath = os.path.join(data_dir, "train/subj{0}_series{1}_data.csv".format(subject, s))
        data.append(pd.read_csv(data_filepath, index_col=0).values.astype(np.float))
        events_filepath = os.path.join(data_dir, "train/subj{0}_series{1}_events.csv".format(subject, s))
        events.append(pd.read_csv(events_filepath, index_col=0).values)
    return data, np.concatenate(events, axis=0)

In [3]:
data, events = load_dataset(subject=1, data_dir='data')

Load series 1 from patient 1
Load series 2 from patient 1
Load series 3 from patient 1
Load series 4 from patient 1
Load series 5 from patient 1
Load series 6 from patient 1
Load series 7 from patient 1
Load series 8 from patient 1


### Low-pass filtering <a class="anchor" id="low-pass-filtering"></a>

To avoid creating potential aliasing effects, we apply necessary spectral filters on the raw signals before to downsample them.

In [4]:
class LowPassFilter:
    
    def __init__(self, cutoff_freq, sampling_freq, order=3):
        self.nyquist_freq = sampling_freq / 2.
        bound = cutoff_freq / self.nyquist_freq
        self.b, self.a = scipy.signal.butter(order, bound, btype='lowpass', analog=True)
    
    def filter(self, signal):
        # TODO: does lfiter include future data when filtering current frame?
        return scipy.signal.lfilter(self.b, self.a, signal, axis=0)

In [5]:
Fp1 = [series[:, 0] for series in data] # TODO: electrode 0 or 15
filtered = list()
for cutoff_freq in np.linspace(0, 1, 11)[1:]:
    lowpass = LowPassFilter(cutoff_freq, 500)
    filtered.append(np.concatenate([lowpass.filter(series_fp1) for series_fp1 in Fp1], axis=0)[:, np.newaxis])
filtered = np.concatenate(filtered, axis=1)
X_train_raw = np.concatenate((np.concatenate(data, axis=0), filtered, filtered ** 2), axis=1)

### Downsampling <a class="anchor" id="downsampling"></a>

In [6]:
X_train_downsampled = X_train_raw[::4000]
y_downsampled = events[::4000]

In [7]:
X_train_downsampled = StandardScaler(copy=False).fit_transform(X_train_downsampled)

In [8]:
X_train, X_val = X_train_downsampled[:200], X_train_downsampled[200:]
y_train, y_val = y_downsampled[:200], y_downsampled[200:]

## A first naive approach: learn from EEG samples <a class="anchor" id="learn-from-raw-data"></a>
<hr/>

### Random forests <a class="anchor" id="random-forests"></a>

The classical decision tree algorithms suggest to grow a tree by splitting each node according to the attribute and the split value that jointly minimize a
given cost function. The most commonly used cost functions are the impurity (also called GINI) and the Shannon entropy.
It has been shown that in practice such method may lead to overfitting problems. Instead, the model introduced by [Leo Breiman](#leo-breiman) tends to decrease its generalization error by both combining predictions of independent tree classifiers and maintaining a good individual 
performance for each of them. The random forest algorithm grows individual trees while respecting several properties:

* **Random feature selection**: when splitting a node into two or more children, one must considerate a random subset of attributes used to evaluate the cost function, instead of testing each one of them. The subset size is a hyper-parameter that has to be decided empirically.
* **Bagging**: Bagging (or bootstrap aggregation) is the fact of fitting each tree of the forest with a subset of the original training set with replacement. Random forests use bagging to both minimize the generalization error (this is possible by jointly randomizing feature selection and using bootstrap aggregation/bagging) and making **out-of-bag estimates** for the generalization error. In order to make out-of-bag estimates, a new classifier (called out-of-bag classifier) is build by combining, for each training instance $X_i$, the predictions made by the trees that did not have $X_i$ in their bagging samples. Hence, only a subset of the trees are used when computing the error related to one instance.
* Grown trees are **not pruned** using any post-pruning algorithm.

#### VC-dimension of random forests <a class="anchor" id="random-forests-vc"></a>

Let's denote $s$ the strength of the set of classifiers $\{ h(x, \Theta) \}$, where $h$ is the random forest prediction function,
$x$ is an input instance, and $\Theta$ is the set of parameters of the model (node children, attribute identifiers and split values).

\begin{equation}
  s = \mathop{\mathbb{E}}_{X, Y} P_{\Theta}(h(X, \Theta) = Y) - \max_{j \neq Y} P_{\Theta}(h(X, \Theta) = j)
\end{equation}

The raw margin function is given by:

\begin{equation}
  rmg(\Theta, X, Y) = argmax_{j \neq Y} P_{\Theta} (h(X, \Theta) = j)
\end{equation}

$\bar{\rho}$ is the mean value of the correlation $\rho(\Theta, \Theta')$ between $rmg(\Theta, X, Y)$ and $rmg(\Theta', X, Y)$ 
holding $\Theta, \Theta'$ fixed.

An upper bound for the generalization error is given by the following inequality:

\begin{equation}
  PE^{*} \le \bar{\rho} (1 - s^2) / s^2
\end{equation}

<span style="color:red">TODO: explanations about the formulas, explanation on how to use oob score to estimate the 
generalization error and some python code to compute a bound down below, eventually.</span>

In [14]:
for i in range(6):
    rf = RandomForestClassifier(n_estimators=150, criterion="entropy", oob_score=True)
    rf.fit(X_train, y_train[:, i])
    scores = rf.predict_proba(X_val)[:, 1]
    oob_error = 1. - rf.oob_score_
    print("Fit model %i on %s..." % (i+1, EVENT_NAMES[i]))
    print("\tROC AUC score: %f" % roc_auc_score(y_val[:, i], scores))
    print("\tOut-of-bag error: %f\n" % oob_error)

Fit model 1 on HandStart...
	ROC AUC score: 0.540296
	Out-of-bag error: 0.005000

Fit model 2 on FirstDigitTouch...
	ROC AUC score: 0.659864
	Out-of-bag error: 0.045000

Fit model 3 on BothStartLoadPhase...
	ROC AUC score: 0.662162
	Out-of-bag error: 0.040000

Fit model 4 on LiftOff...
	ROC AUC score: 0.397891
	Out-of-bag error: 0.035000

Fit model 5 on Replace...
	ROC AUC score: 0.551948
	Out-of-bag error: 0.025000

Fit model 6 on BothReleased...
	ROC AUC score: 0.764610
	Out-of-bag error: 0.015000



#### Random forest pruning <a class="anchor" id="random-forest-pruning"></a>

In [ ]:
LEAF_NODE = -1

def post_pruning(tree, min_samples_leaf=1):
    if tree.min_samples_leaf < min_samples_leaf:
        tree.min_samples_leaf = min_samples_leaf
        tree_ = tree.tree_
        for i in range(tree_.node_count):
            n_samples = tree_.n_node_samples[i]
            if n_samples <= min_samples_leaf:
                tree_.children_left[i] = LEAF_NODE
                tree_.children_right[i] = LEAF_NODE

### Logistic regression <a class="anchor" id="logistic-regression"></a>

Logistic regression is a linear model used for classification. Contrary to a linear classifier obtained with the 
Perceptron Learning Algorithm (PLA), it does not output a binary decision given an instance but real-valued predictions that can be viewed as its degree of confidence that the instance belongs to the positive class. These real values are comprised between
0 and 1 and be interpreted as the probability, for each instance, of belonging to the positive class.
Like in linear regression, the input vector $x_i$ of instance $i$ is projected to a scalar $s_i$ called the "signal".

\begin{equation}
  s_i = w^T x_i + c
\end{equation}

where c is the intercept. 
However, nothing ensures that $s_i$ is in $[0, 1]$. Thus, a bounded function called the sigmoid function is 
applied to $s_i$ to fulfill this goal:

\begin{equation}
  h(x_i) = \sigma(s_i) = \frac{1}{1 + e^{-w^T x_i + c}} = \frac{e^{w^T x_i + c}}{1 + e^{w^T x_i + c}}
\end{equation}

This is more convenient because in most machine learning application one wishes to get a warning of the model about prediction
uncertainty. The model ability to make output continuous intermediate values between a 100%-confident positive prediction
and a 100%-confident negative prediction is referred to as soft thresholding.

Let's use the default optimization algorithm from scikit-learn to fit our logistic regressor. The parameters of the model
are penalized by a L2-regularization. Thus, the optimization problem reduces to minimizing the following cost function:

\begin{equation}
  min_w L(w) = min_w \frac{1}{2} w^T w + C \cdot \Sigma_{i=1}^n \log(e^{-y_i (x_i^T w + c)} + 1)
\end{equation}

where parameter $C$ is set to 1 by default, $y_i$ is the ground-truth label $\in \{-1, 1\}$ associated to instance $i$,
and $\frac{1}{2} w^T w$ is the regularization term. This function is better suited for our model than accuracy because it
does take the model uncertainty into account when assigning a loss on a particular instance. The default scikit-learn
optimization algorithm for logistic regression is based on Coordinate Descent (CD). CD can be described in a few steps:

> **1.** Choose an initial weight vector $w^0$. Let $w_i^0$ be the ith component of $w^0$.

> **2.** Repeat until stop condition is met

>>> **3.** Repeat with $i \in \{1, \ldots, n\}$

>>> **4.** $w_i^{k+1} = argmin_{y} \ L(w_1^{k+1}, \ldots, w_{i-1}^{k+1}, y, w_{i+1}^{k}, \ldots, w_{n}^{k})$

>>>This corresponds to a line search along axis $i$.

>> **4.** $k \leftarrow k+1$

In brief, we observe that the algorithm optimizes the cost function according to a single dimension at a time, by performing
a line search along it. This requires, in average, much more step than gradient descent or quasi-newtonian methods since
CD has no information about the direction of the steepest descent.

#### VC dimension of logistic regression <a class="anchor" id="logistic-regression-vc"></a>

<span style="color:red">TODO: I guess that the VC dimension of logreg is the same as the VC dimension
of the linear classifier since they both shatter the data points in the same way, but the VC bounds must be
different since the error metric is different -> do some research.</span>

In [ ]:
for i in range(6):
    logreg = LogisticRegression(C= 1., fit_intercept=True)
    logreg.fit(X_train, y_train[:, i])
    proba = logreg.predict_proba(X_val)[:, 1]

### Extract covariance matrices <a class="anchor" id="extract-cov"></a>

In [7]:
%%cython

import numpy as np
cimport numpy as cnp
cnp.import_array()
import cython

@cython.boundscheck(False)
@cython.overflowcheck(False)
def extract_cov_matrices(cnp.float_t[:, :] data, Py_ssize_t w):
    cdef Py_ssize_t n_features = data.shape[1]
    cdef cnp.float_t[:, :, :] sigmas = np.empty((data.shape[0], n_features, n_features), dtype=np.float)
    cdef cnp.float_t[:] means = np.asarray(np.mean(np.asarray(data)[:w, :], axis=0), dtype=np.float)
    cdef cnp.float_t[:] old_means = np.copy(means)
    cdef cnp.float_t[:, :] last_sigma = np.cov(np.asarray(data)[:w, :].T)
    cdef cnp.float_t c
    np_sigmas = np.asarray(sigmas)
    np_sigmas[:w, :, :] = np.repeat(np.asarray(last_sigma).reshape((1, n_features, n_features), order='C'), w, axis=0)
    cdef Py_ssize_t i, j, a, b
    with nogil:
        for i in range(w, data.shape[0]):
            for a in range(n_features):
                old_means[a] = means[a]
                means[a] += (data[i, a] - data[i-w, a]) / w
            for a in range(n_features):
                for b in range(a+1):
                    c = sigmas[i-1, a, b]
                    c += (data[i, a] * data[i, b] - data[i-w, a] * data[i-w, b]) / w
                    c += old_means[a] * old_means[b] - means[a] * means[b]
                    sigmas[i, a, b] = sigmas[i, b, a] = c
    return np_sigmas

## Riemannian-based kernel trick <a class="anchor" id="kernel-trick"></a>

In [56]:
def hlmap(Cp, sqrtCinv):
    return scipy.linalg.logm(sqrtCinv * Cp * sqrtCinv)

def hemap(Cp, sqrtC):
    return scipy.linalg.expm(sqrtC * Cp * sqrtC)

def project_cov_matrices(X):
    """ X is of shape (n_samples, n_electrodes, n_electrodes) """
    sqrtC = scipy.linalg.sqrtm(X.mean(axis=0))
    sqrtCinv = scipy.linalg.inv(sqrtC)
    return np.asarray([hlmap(h, sqrtCinv) for h in X])

### Riemannian-based Support Vector Machine <a class="anchor" id="svm"></a>

<span style="color:red">TODO</span>

### Linear Discriminant Analysis <a class="anchor" id="lda"></a>

<span style="color:red">TODO</span>

## Dealing with non-stationary processes <a class="anchor" id="non-stationary-data"></a>
<hr/>

### Gaussian Hidden Markov Models (G-HMM) <a class="anchor" id="ghmm"></a>

<span style="color:red">TODO</span>

## Bibliography <a class="anchor" id="bibliography"></a>
<hr/>

* <span class="anchor" id="bib-riemann">
    [1] Classification of covariance matrices using a Riemannian-based kernel for BCI applications <br>
    Alexandre Barachant, Stéphane Bonnet, Marco Congedo, Christian Jutten <br>
    https://hal.archives-ouvertes.fr/file/index/docid/820475/filename/BARACHANT_Neurocomputing_ForHal.pdf <br>
  </span>
<br>

* <span class="anchor" id="bib-kaggle">
    [2] Grasp-and-Lift EEG Detection Kaggle Competition <br>
    https://www.kaggle.com/c/grasp-and-lift-eeg-detection <br>
  </span>
<br>

* <span class="anchor" id="leo-breiman">
    [3] Random Forests <br>
    Leo Breiman, 2001 <br>
    https://link.springer.com/content/pdf/10.1023%2FA%3A1010933404324.pdf <br>
  </span>
<br>